In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam


In [14]:
data = {
    'income': np.random.randint(20000, 100000, 1000),
    'debt_to_income_ratio': np.random.uniform(0.1, 1.0, 1000),
    'credit_utilization': np.random.uniform(0.1, 1.0, 1000),
    'payment_history': np.random.uniform(0, 1, 1000),
    'age': np.random.randint(18, 75, 1000),
    'credit_rating': np.random.randint(300, 850, 1000),
}

In [15]:
df = pd.DataFrame(data)

In [16]:
X = df[['income', 'debt_to_income_ratio', 'credit_utilization', 'payment_history', 'age']]
y = df['credit_rating'] / 850 


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
model = Sequential([
    Dense(256, input_dim=X_train.shape[1], activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(1, activation='linear')  # Linear activation for regression
]) 

c:\Users\keith\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mean_absolute_error'])


In [21]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=1)

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - loss: 1.9388 - mean_absolute_error: 1.0974 - val_loss: 0.3051 - val_mean_absolute_error: 0.5117
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.1745 - mean_absolute_error: 0.8558 - val_loss: 0.1993 - val_mean_absolute_error: 0.3971
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.8037 - mean_absolute_error: 0.7081 - val_loss: 0.1332 - val_mean_absolute_error: 0.3066
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.6486 - mean_absolute_error: 0.6475 - val_loss: 0.0903 - val_mean_absolute_error: 0.2451
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.5612 - mean_absolute_error: 0.6013 - val_loss: 0.0596 - val_mean_absolute_error: 0.2035
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4672 - mean_absolute_error: 0.5340 - val_loss: 0.0546 - val_mean_absolute_error: 0.1957
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.4335 - mean_absolute_error: 0.5128 - val_loss:

In [22]:
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Mean Absolute Error: {mae:.2f}")


Test Mean Absolute Error: 0.17


In [23]:
new_data = np.array([[50000, 0.3, 0.4, 0.9, 35]])  
new_data_scaled = scaler.transform(new_data)
predicted_rating = model.predict(new_data_scaled)
print(f"Predicted Credit Rating (scaled): {predicted_rating[0][0]:.2f}")
print(f"Predicted Credit Rating (actual): {predicted_rating[0][0] * 850:.0f}")

c:\Users\keith\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step
Predicted Credit Rating (scaled): 0.61
Predicted Credit Rating (actual): 521


In [24]:
train_data = pd.DataFrame(X_train, columns=X.columns)
train_data['credit_rating'] = y_train
train_data.to_csv("train_data.csv", index=False)

test_data = pd.DataFrame(X_test, columns=X.columns)
test_data['credit_rating'] = y_test
test_data.to_csv("test_data.csv", index=False)
